IMPORT LIBRARIES

In [2]:
import os

In [2]:
# parameter definition
DIR = "/home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images"
DIR_GPS = "/home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/trou_de_sable_1_2021-06-12/log_gps_reach"
llh_file = "solution_202106120828.LLH"
llh_path = os.path.join(DIR_GPS, llh_file)
# N.B. insert the date and time following the format "YYYY:MM:DD HH:MM:SS.000"
my_time = "2021:06:12 08:28:22.000"

In [8]:
def llh_to_txt(llh_path) :
    # Inputs :
    # 1.llh_path = path of the llh file 
    
    # Outputs :
    # 1.txt_path = path of the txt file, which can be used by exiftool functions
    # The function saves a txt file with the same information of the LLH file in the same directory
    
    # open LLH file 
    f = open(llh_path,'r')
    # get LLh file name and replace .LLH by .txt
    llh_file_name = os.path.basename(os.path.normpath(llh_path))
    txt_file_name = llh_file_name.replace("LLH", "txt")
    txt_path = llh_path.replace(llh_file_name, txt_file_name)
    # open txt file
    newf = open(txt_path, 'w')
    # read old content
    lines = f.readlines() 
    # write new line with all variables
    newf.write("GPSDateStamp,GPSTimeStamp,GPSLatitude,GPSLongitude,elevation,fix,nbsat,sdn,sde,sdu,sdne,sdeu,sdun,age,ratio \n") # write new content at the beginning
    # write old content after new
    for line in lines: 
        # replace every kind of space with a comma
        line = line.replace("     ", ",")
        line = line.replace("    ", ",")
        line = line.replace("   ", ",")
        line = line.replace("  ", ",")
        line = line.replace(" ", ",")
        newf.write(line)
    # close files
    newf.close()
    f.close()
    return txt_path

In [9]:
def time_calibration_and_geotag(DIR, my_time, txt_path) :
    # Inputs :
    # 1.DIR = images directory
    # 2.my_time = date and time following the format "YYYY:MM:DD HH:MM:SS.000"
    # 3.txt_path = path of the txt file, which can be used by exiftool functions
    
    # Outputs :
    # The function adds all the metadata contained in the LLH file into images
  
    # see :
    # https://exiftool.org/forum/index.php?topic=7033.0
    change_date_and_time =  "exiftool -m '-SubSecDateTimeOriginal= " + my_time + "' " + DIR + " -fileorder filename -overwrite_original"
    os.system(change_date_and_time)
    print_date_and_time = "exiftool -m -SubSecDateTimeOriginal " + DIR + " -fileorder filename"
    os.system(print_date_and_time)
    # First, copy SubSecDateTimeOriginal to XMP:DateTimeOriginal
    change_tag_date_and_time = "exiftool -m '-XMP:DateTimeOriginal<SubSecDateTimeOriginal' " + DIR + " -fileorder filename -overwrite_original"
    os.system(change_tag_date_and_time)
    # Then, run your microsecond change on the XMP:DateTimeOriginal.  This works because XMP timestamps are more flexible than EXIF timestamps.
    increment_time = "exiftool -m '-XMP:DateTimeOriginal+<0:0:${filesequence;$_*=0.200}' " + DIR + " -fileorder filename -overwrite_original"
    os.system(increment_time)
    print_date_and_time = "exiftool -m -XMP:DateTimeOriginal " + DIR + " -fileorder filename"
    os.system(print_date_and_time)
    # Finally, copy the XMP:DateTimeOriginal to the other timestamps.
    update_date_and_time =  "exiftool -m '-SubSecTimeOriginal<XMP:DateTimeOriginal'  '-SubSecTime<XMP:DateTimeOriginal' '-SubSecTimeDigitized<XMP:DateTimeOriginal' '-datetimeoriginal<XMP:DateTimeOriginal' " + DIR + " -fileorder filename -overwrite_original"
    os.system(update_date_and_time)
    # Copy gps informations from DIR_GPS to images metadata
    write_gps = "exiftool -m  -geotag " + txt_path + " '-geotime<${XMP:DateTimeOriginal}+00:00' " + DIR + " -overwrite_original "
    #exiftool -geotag trk.gpx '-geotime<${DateTimeOriginal}+00:00' dir
    os.system(write_gps)

In [10]:
txt_path = llh_to_txt(llh_path)

'/home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/trou_de_sable_1_2021-06-12/log_gps_reach/solution_202106120828.txt'

In [11]:
time_calibration_and_geotag(DIR, my_time, txt_path)

    1 directories scanned
    5 image files updated
======== /home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images/session_2021_01_13_Hermitage_AllRounder_image_000146.jpg
Date/Time Original              : 2021:06:12 08:28:22.000
======== /home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images/session_2021_01_13_Hermitage_AllRounder_image_000156.jpg
Date/Time Original              : 2021:06:12 08:28:22.000
======== /home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images/session_2021_01_13_Hermitage_AllRounder_image_000166.jpg
Date/Time Original              : 2021:06:12 08:28:22.000
======== /home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images/session_2021_01_13_Hermitage_AllRounder_image_000186.jpg
Date/Time Original              : 2021:06:12 08:28:22.000
======== /home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images/session_2021_01_13_Hermitage_AllRounder_image_000196.jpg
Date/Time Original              : 2021:06:12 08:28:22.000
   

In [3]:
# PRINT GPS INFORMATIONS FOR ALL FILES IN FOLDER
DIR = "/home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images"
#os.system("exiftool -time:all -s " + DIR)
images = os.listdir(DIR)
images = sorted(images)
for img in images :
    print("##################")
    print(img)
    path_img = os.path.join(DIR, img)
    os.system("exiftool -n " + path_img)

##################
session_2021_01_13_Hermitage_AllRounder_image_000146.jpg
ExifTool Version Number         : 12.40
File Name                       : session_2021_01_13_Hermitage_AllRounder_image_000146.jpg
Directory                       : /home/mcontini/Desktop/Ifremer/Plancha/datarmor_code/test_images
File Size                       : 532622
File Modification Date/Time     : 2022:04:12 10:35:22+04:00
File Access Date/Time           : 2022:04:12 10:37:00+04:00
File Inode Change Date/Time     : 2022:04:12 10:35:22+04:00
File Permissions                : 100664
File Type                       : JPEG
File Type Extension             : JPG
MIME Type                       : image/jpeg
Exif Byte Order                 : MM
X Resolution                    : 72
Y Resolution                    : 72
Resolution Unit                 : 2
Y Cb Cr Positioning             : 1
Exif Version                    : 0232
Date/Time Original              : 2021:06:12 08:28:22
Components Configuration        : 